In [269]:
# Para llevar el registro
!pip install -q comet_ml
import comet_ml
comet_ml.init(project_name = 'FineTunning')

In [270]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input, Conv2D, MaxPooling2D, Concatenate, PReLU
from tensorflow.keras.optimizers import RMSprop, SGD, Adam, Adadelta
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.regularizers import l2

In [271]:
# Se crea el experimento en Comet
experiment = comet_ml.Experiment(
    auto_histogram_weight_logging = True,
    auto_histogram_gradient_logging = True,
    auto_histogram_activation_logging = True,
    log_code = True
)
experiment.set_name('Test 29')

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gurumasterahb/finetunning/849a25cb644241528f2d6d3e9a886b5c
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [91]                : (0.3499999940395355, 0.5249999761581421)
COMET INFO:     batch_accuracy [92]          : (0.25, 0.625)
COMET INFO:     batch_loss [92]              : (0.7428814768791199, 30.27588653564453)
COMET INFO:     epoch_duration [91]          : (2.7490920899981575, 6.18331226599912)
COMET INFO:     loss [91]                    : (0.7408625483512878, 16.16670036315918)
COMET INFO:     val_accuracy [91]            : (0.44999998807907104, 0.5249999761581421)
COMET INFO:     val_loss [91]        

In [272]:
# Parámetros
i_width = int(178 * 1.2)
i_height = int(218 * 1.2)
input_shape = (i_width, i_height, 3)

parameters = {
    'num_class' : 2,
    'epochs' : 350,
    'batch_size' : 32,
    'learning_rate' : 0.001,
    'loss' : 'binary_crossentropy',
    'optimizer' : 'RMSprop'}

experiment.log_parameters(parameters)

In [273]:
# Loading images
train_dir = '/kaggle/input/data-finetunning/Norma_align/Norma_align/train'
val_dir = '/kaggle/input/data-finetunning/Norma_align/Norma_align/val'

# Image Generator
gen_train = ImageDataGenerator(
    rotation_range = 30,
    width_shift_range = 0.15,  # % de desplazamiento horizontal
    height_shift_range = 0.15, # % de desplazamiento vertical
    brightness_range = [0.25, 1.25], # Porcentaje de brillo en el rango
    zoom_range = 0.3, # Zoom de hasta 30%
    channel_shift_range = 20, # Cada pixel de cada canal varía su intensidad en una vecindad de 20 unidades
    fill_mode = 'nearest', # Después de desplazamientos que dejan la imágen 'incompleta' se rellena con el valor de los pixeles más cercanos
    horizontal_flip = True,
    vertical_flip = True,
    rescale = 1./255.)
train = gen_train.flow_from_directory(
    train_dir,
    target_size = (i_width, i_height),
    color_mode = 'rgb',
    class_mode = 'binary',
    batch_size = parameters['batch_size'],
    shuffle = True)

gen_val = ImageDataGenerator(
    rotation_range = 30,
    width_shift_range = 0.15,
    height_shift_range = 0.15,
    brightness_range = [0.25, 1.25],
    zoom_range = 0.3,
    channel_shift_range = 20,
    fill_mode = 'nearest',
    horizontal_flip = True,
    vertical_flip = True,
    rescale = 1./255.)
val = gen_val.flow_from_directory(
    val_dir,
    target_size = (i_width, i_height),
    color_mode = 'rgb',
    class_mode = 'binary',
    batch_size = parameters['batch_size'],
    shuffle = True)

Found 120 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [274]:
# Loading model
path_load = os.path.join('/kaggle/input/data-finetunning/best_val_loss.h5')
model_loaded = tf.keras.models.load_model(path_load)

model_0 = Sequential()

for layer in model_loaded.layers[:15]:
    layer.trainable = False
    model_0.add(layer)

# model_0.add(Dense(128, activation = 'relu', kernel_regularizer = l2(0.01)))
model_0.add(Dense(512, activation = 'PReLU'))
model_0.add(Dropout(0.4))
model_0.add(Dense(256, activation = 'PReLU'))
model_0.add(Dropout(0.12))
model_0.add(Dense(128, activation = 'relu', kernel_regularizer = l2(0.1)))
model_0.add(Dropout(0.5))
model_0.add(Dense(64, activation = 'relu', kernel_regularizer = l2(0.15)))
model_0.add(Dense(1, activation = 'sigmoid'))
    
# model_0.summary()

In [275]:
# Checkpoint para el mejor modelo (val_accuracy)
filepath = 'best_val_loss.h5'
checkpoint = ModelCheckpoint(
    filepath,
    monitor = 'val_loss',
    verbose = 1,
    save_best_only = True,
    mode = 'min')

In [276]:
# Compilación y entrenamiento
model_0.compile(
    loss = parameters['loss'],
    optimizer = RMSprop(learning_rate = parameters['learning_rate']),
    metrics = ['accuracy'])

model_0.fit(
    train,
    epochs = parameters['epochs'],
    verbose = 1,
    validation_data = val,
    callbacks = [checkpoint])

Epoch 1/350
4/4 [==============================] - ETA: 0s - loss: 28.1975 - accuracy: 0.5000
Epoch 1: val_loss improved from inf to 25.26152, saving model to best_val_loss.h5


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4/4 [==============================] - 6s 1s/step - loss: 28.1975 - accuracy: 0.5000 - val_loss: 25.2615 - val_accuracy: 0.5750
Epoch 2/350
4/4 [==============================] - ETA: 0s - loss: 24.1541 - accuracy: 0.4917
Epoch 2: val_loss improved from 25.26152 to 22.49712, saving model to best_val_loss.h5
4/4 [==============================] - 4s 960ms/step - loss: 24.1541 - accuracy: 0.4917 - val_loss: 22.4971 - val_accuracy: 0.6750
Epoch 3/350
4/4 [==============================] - ETA: 0s - loss: 21.6022 - accuracy: 0.5167
Epoch 3: val_loss improved from 22.49712 to 20.33846, saving model to best_val_loss.h5
4/4 [==============================] - 4s 1s/step - loss: 21.6022 - accuracy: 0.5167 - val_loss: 20.3385 - val_accuracy: 0.6000
Epoch 4/350
4/4 [==============================] - ETA: 0s - loss: 19.6043 - accuracy: 0.6083
Epoch 4: val_loss improved from 20.33846 to 18.55211, saving model to best_val_loss.h5
4/4 [==============================] - 4s 1s/step - loss: 19.6043 - ac

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4/4 [==============================] - 3s 913ms/step - loss: 7.5990 - accuracy: 0.7417 - val_loss: 7.1110 - val_accuracy: 0.7000
Epoch 15/350
4/4 [==============================] - ETA: 0s - loss: 6.8828 - accuracy: 0.7333
Epoch 15: val_loss improved from 7.11104 to 6.44721, saving model to best_val_loss.h5
4/4 [==============================] - 4s 957ms/step - loss: 6.8828 - accuracy: 0.7333 - val_loss: 6.4472 - val_accuracy: 0.6750
Epoch 16/350
4/4 [==============================] - ETA: 0s - loss: 6.1644 - accuracy: 0.7083
Epoch 16: val_loss improved from 6.44721 to 5.80655, saving model to best_val_loss.h5


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4/4 [==============================] - 4s 1s/step - loss: 6.1644 - accuracy: 0.7083 - val_loss: 5.8065 - val_accuracy: 0.6500
Epoch 17/350
4/4 [==============================] - ETA: 0s - loss: 5.5574 - accuracy: 0.7583
Epoch 17: val_loss improved from 5.80655 to 5.21741, saving model to best_val_loss.h5
4/4 [==============================] - 3s 898ms/step - loss: 5.5574 - accuracy: 0.7583 - val_loss: 5.2174 - val_accuracy: 0.6000
Epoch 18/350
4/4 [==============================] - ETA: 0s - loss: 4.9682 - accuracy: 0.7333
Epoch 18: val_loss improved from 5.21741 to 4.67778, saving model to best_val_loss.h5
4/4 [==============================] - 3s 960ms/step - loss: 4.9682 - accuracy: 0.7333 - val_loss: 4.6778 - val_accuracy: 0.7250
Epoch 19/350
4/4 [==============================] - ETA: 0s - loss: 4.4263 - accuracy: 0.7583
Epoch 19: val_loss improved from 4.67778 to 4.11596, saving model to best_val_loss.h5
4/4 [==============================] - 3s 904ms/step - loss: 4.4263 - accura

In [277]:
experiment.log_model("Facial_r", filepath)
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gurumasterahb/finetunning/a048b4d60e3b47778b4fdaf50881e541
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [350]                : (0.49166667461395264, 0.8916666507720947)
COMET INFO:     batch_accuracy [350]          : (0.375, 1.0)
COMET INFO:     batch_loss [350]              : (0.20243141055107117, 30.438140869140625)
COMET INFO:     epoch_duration [350]          : (2.7396827599986864, 6.268242427999212)
COMET INFO:     loss [350]                    : (0.35633614659309387, 28.197452545166016)
COMET INFO:     val_accuracy [350]            : (0.5, 0.949999988079071)
COMET INFO:     val_loss [350]             